# Discussion 1

In this discussion we review vector and matrix operations by focusing on their computational aspects. 

You can use the Shared Computing Cluster (SCC) or Google Colab to run this notebook.

The general instructions for running on the SCC are available under General Resources on [Piazza](https://piazza.com/bu/fall2025/ds722/resources).

## Formulas

### Inner Product

The formula for the inner product of two vectors $x,y \in \mathbb{R}^n$ is given by 
$$
x^Ty = \sum_{i=1}^n x_i y_i.
$$

### Matrix Vector Multiplication

The formula for the matrix-vector product $y = Ax$ where $A \in \mathbb{R}^{m \times n}$ and $x \in \mathbb{R}^n$ is given by
$$
y_i = \sum_{j=1}^n a_{ij} x_j,
$$
for $i=1,\ldots,m$.

### Matrix Matrix Multiplication

The formula for the matrix-matrix product $C = AB$ where $A \in \mathbb{R}^{m \times n}$ and $B \in \mathbb{R}^{n \times p}$ is given by
$$
c_{ij} = \sum_{k=1}^n a_{ik} b_{kj},
$$
for $i=1,\ldots,m$ and $j=1,\ldots,p$.

# Problem 1 - FLOPS

First compute by hand the number of flops (floating point operations) for each of the following:

- inner product of two vectors
- matrix-vector multiplication
- matrix-matrix multiplication

A flop is a single arithmetic operation involving floating-point numbers such as:

- addition
- subtraction
- multiplication
- division

For this problem you are essentially counting the total number of multiplications and divisions.

## Inner product

Given two vectors $x,y\in\mathbb{R}^{n}$ what is the flop count of the inner product?

## Matrix-vector product

Given a matrix $A\in\mathbb{R}^{m \times n}$ and vector $x\in\mathbb{R}^{n}$ what is the flop count for the product $y=Ax$?

## Matrix-Matrix product

Given a matrix $A\in\mathbb{R}^{m\times n}$ and matrix $B\in\mathbb{R}^{n\times p}$ what is the flop count for the product $C=AB$?

# Problem 2 - Computational Complexity

Using NumPy arrays, code your own implementations of

- the inner product of two vectors, `inner_product(a, b)`
- matrix-vector multiplication, `matrix_vector_product(A, x)`
- matrix-matrix multiplication, `matrix_vector_product(A, B)`

In addition to returning the relevant computed quantity, calculate the number of flops in your routine. Verify that this number matches what you calculated by hand.

In [ ]:
#TODO

In [ ]:
# TODO

In [ ]:
# TODO

In [ ]:
# Tests

import numpy as np
rng = np.random.default_rng(seed=42)

x = rng.random(5)
y = rng.random(5)

inner_product_result, inner_flops = inner_product(x, y)

print(inner_flops) # Expect 9


In [ ]:
A = rng.random((10, 5))

b, mv_flops = matrix_vector_product(A, x)

print(mv_flops) # Expect 90

In [ ]:
B = rng.random((5, 10))

C, mm_flops = matrix_matrix_product(A, B)

print(mm_flops) # Expect 900

# Problem 3 - A closer look at Matrix-Matrix multiplication

Recall the definition of matrix multiplication. Given $A\in\mathbb{R}^{m \times n}$, $B\in\mathbb{R}^{n \times p}$, the matrix-matrix product $C = AB \in\mathbb{R}^{m \times p}$ is defined as

$$
c_{ij} = \sum_{k=1}^{n} a_{ik}b_{kj},
$$

for $i=1,\ldots,m$ and $j=1,\ldots,p$.

In your previous code you most likely implemented this as 3 nested loops, where $i$ ranged overed the $m$ rows in $A$, $j$ ranged over the $p$ columns of $B$ and $k$ ranged over the $n$ columns(rows) of $A$($B$). This $ijk$ loop ordering corresponds to inner loop data access of $A$ by row and $B$ by column.

There is no reason we cannot re-order these loops. There are in fact six possible orderings. The total number of flops stays the same for each ordering, but the pattern of data access changes. Let's investigate a few of these.

## Column Partitions
In class we discussed matrix-matrix multiplication as the linear combination of columns of $A$ with coefficients coming from the columns of $B$. This corresponds to an $ikj$ loop ordering.

Write a routine `matrix_matrix_columns(A, B)` which uses columns to implement the matrix multiplication, i.e., your function should use `A[:,k] * B[k, j]`. This function should only have 2 nested loops. You don't need to return the number of flops, only the result of multiplying matrices $A$ and $B$. Verify that this returns the same values

In [ ]:
#TODO

In [ ]:
C2 = matrix_matrix_columns(A, B)

np.array_equal(C, C2) # Expect True

Observe that by vectorizing the $i$ loop over $m$, we are computing the columns of $C$ as linear combinations of the columns of $A$ with coefficients from the columns of $B$.


## Outer Product

One of your homework problems is to show that matrix-matrix multiplication can be computed as a sum of outer products -- so this might be a helpful exercise.

Write a routine `matrix_matrix_outer(A, B)` which uses an outer product
You can partition $A$ into columns and $B$ into rows and use `np.outer(A[:,k], B[k, :])` to represent the outer product.

In [ ]:
#TODO

In [ ]:
C3 = matrix_matrix_outer(A, B)

np.array_equal(C, C3) # Expect True

# Problem 4 - When dimensions get large

For this exercise we consider $A,B\in\mathbb{R}^{m\times m}$. From the previous exercise we know that matrix-matrix multiplication $C=AB$ is an $\mathcal{O}(m^{3})$ operation, i.e., if we double the size of our matrix, the number of flops increases by a factor of $8$.

For this problem, let's compare the routines that we wrote along with the default NumPy implementation of matrix-matrix multiplication using the `@` operator. 

Specifically, write a loop ranging over `m = [10, 20, 40, 80, 160, 320, 640]` creating random matrices $A,B\in\mathbb{R}^{m\times m}$ of and time each of the following routines

- `matrix_matrix_product(A, B)`
- `matrix_matrix_columns(A, B)`
- `matrix_matrix_outer(A, B)`
- `A @ B`

Plot the times on a semilog axis. Does the plot confirm the expected $\mathcal{O(m^{3})}$ work?

In [ ]:
#TODO

# Problem 5 - Hardware Acceleration (Optional)

For this problem you should use the SCC or Colab with a GPU. For those of you using the SCC, I advise working on this problem in a group so that you get a GPU faster.

Matrix multiplication can benefit from hardware acceleration using a Graphics Processing Units (GPU). GPUs are particularly well-suited for this task due to their architecture and capabilities. The reasons are

- The inner products in matrix multiplication:

$$
  C_{ij} = \sum_{k=1}^{n} A_{ik}B_{kj},
$$

can be done **in parallel**. GPUs have thousands of cores that can perform these operations simultaneously, unlike CPUs which typically have fewer cores.

- GPUs are compute a much larger number of flops/s than CPUs.

- High memory bandwith and optimized memory access patterns.

Complete the following code cell using that uses a GPU to multiply two matrices using Pytorch. How does this time compare to the times on a CPU? 

Print these times and confirm if the asymptotic complexity is still $\mathcal{O}(m^{3})$.

In [ ]:
import torch
from torch.utils.benchmark import Timer

torch.manual_seed(42)

# Benchmarking
sizes = [10, 20, 40, 80, 160, 320, 640, 1280, 2560, 5120, 10240]
times = []

# TODO